In [4]:
import pandas as pd
import requests 
from bs4 import BeautifulSoup
from lxml import etree

In [5]:
names = pd.read_csv("authornames-csv.csv")

In [6]:
names

,Number,Name
0,3,"A., Sonay"
1,4,"Aarestrup, Emil"
2,5,"Abarbanell, Stephan"
3,6,"Abbott, David"
4,7,"Äbdülrähmanlı, Näriman"
...,...,...
4956,4959,"Гейне, Генріх"
4957,4960,"Гете, Иоганн Вольфганг"
4958,4961,"Гримм, Братья"
4959,4962,"Захер-Мазох, Леопольд фон"


In [64]:
#Names to list: 
new = names["Name"].to_list()
shortlist = new[0:250]
supershort = new[0:25]


In [185]:
def solve_problem(entry): 
    if entry.endswith('.'):
        entry = entry.replace(entry, "testestestestestestsetes")
    else:
        entry = entry

new = [solve_problem(x) for x in supershort]
print(new)

[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]


In [161]:
#Umwandeln Liste in Text für Anfrage: 

string = ""
first = True

for item in supershort: 
    if first:
        #string = string + "(" + item + ")"
        string = string + item 
        first = False
    else: 
        #string = string + " OR (" + item + ")"
        string = string + " OR " + item
    
print(string)

searchtext = string
print(searchtext)


#per=((A., Sonay) OR (Abarbanell, Stephan) OR (Abbott, David) OR (Abedi, Isabel) OR (Abel, Winnie)) and bbg=Tp*



A., Sonay OR Aarestrup, Emil OR Abarbanell, Stephan OR Abbott, David OR Äbdülrähmanlı, Näriman OR Abedi, Isabel OR Abel, Winnie OR Abele, Jan OR Abell, Arthur M. OR Abeln, Reinhard OR Ablaß, Werner E. OR Abraham, Filo M. OR Abreu, Carlos OR Acero, Irene OR Achermann, Franz Heinrich OR Achleitner, Friedrich OR Achternbusch, Herbert OR Ackermann, Erich OR Ackermann, Rolf OR Acklin, Jürg OR Acosta, Luis A. OR Adam, Gabi OR Adams Richards, David OR Adams, Lilly OR Addams, Peter
A., Sonay OR Aarestrup, Emil OR Abarbanell, Stephan OR Abbott, David OR Äbdülrähmanlı, Näriman OR Abedi, Isabel OR Abel, Winnie OR Abele, Jan OR Abell, Arthur M. OR Abeln, Reinhard OR Ablaß, Werner E. OR Abraham, Filo M. OR Abreu, Carlos OR Acero, Irene OR Achermann, Franz Heinrich OR Achleitner, Friedrich OR Achternbusch, Herbert OR Ackermann, Erich OR Ackermann, Rolf OR Acklin, Jürg OR Acosta, Luis A. OR Adam, Gabi OR Adams Richards, David OR Adams, Lilly OR Addams, Peter


In [156]:
#Send query to SRU: 
base_url = "https://services.dnb.de/sru/authorities" #for GND
searchtext = "PER="+"Abarbanell, Stephan"+"%20"+"and BBG=Tp*"

parameter = {'version' : '1.1' , 'operation' : 'searchRetrieve' , 'query' : searchtext, 'recordSchema' : "MARC21-xml"}

r1 = requests.get(base_url, params = parameter)
print(r1.url)

response = BeautifulSoup(r1.content)
records = response.find_all('record')

print(response)


https://services.dnb.de/sru/authorities?version=1.1&operation=searchRetrieve&query=PER%3DAbarbanell%2C+Stephan%2520and+BBG%3DTp%2A&recordSchema=MARC21-xml
<?xml version="1.0" encoding="UTF-8"?><html><body><searchretrieveresponse xmlns="http://www.loc.gov/zing/srw/"><version>1.1</version><diagnostics><diag:diagnostic xmlns:diag="http://www.loc.gov/zing/srw/diagnostic/"><diag:uri>info:srw/diagnostic/1/16</diag:uri><diag:details>Unsupported index</diag:details><diag:message>Abarbanell, Stephan%20and BBG</diag:message></diag:diagnostic></diagnostics></searchretrieveresponse></body></html>


In [154]:
#Funktion für SRU-Schnittstelle: 
worklist = []
from tqdm.notebook import tqdm
from time import sleep

i=0
base_url = "https://services.dnb.de/sru/authorities" #for GND
    
for i in tqdm(range(0,len(supershort))):
    #sleep(3)
    
    searchtext = "PER="+supershort[i]+"%20"+"and bbg=Tp*"
    parameter = {'version' : '1.1' , 'operation' : 'searchRetrieve' , 'query' : searchtext, 'recordSchema' : "MARC21-xml"}

    
    r1 = requests.get(base_url, params = parameter)
    #print(r.url)

    #Weiter mit Beautiful Soup
    response = BeautifulSoup(r1.content)
    records = response.find_all('record')

    
    query = supershort[i]
    if records:
        found = "yes"
    else:
        found = "no"
        
    result = {"query":query, "found":found} 
    worklist.append(result)
    
    
    #output = [parse_record(record) for record in records]
    #df = pd.DataFrame(output)
    #worklist.append(df)

    i+=1



  0%|          | 0/25 [00:00<?, ?it/s]

In [155]:
df = pd.DataFrame(worklist)
df

,query,found
0,"A., Sonay",no
1,"Aarestrup, Emil",no
2,"Abarbanell, Stephan",no
3,"Abbott, David",no
4,"Äbdülrähmanlı, Näriman",no
5,"Abedi, Isabel",no
6,"Abel, Winnie",no
7,"Abele, Jan",no
8,"Abell, Arthur M.",yes
9,"Abeln, Reinhard",no
